In [1]:
#this script is to use tranied model and some data points to
#visualize the last "fully-connected layer" of GCN.

import torch
#from GCN_regression import *
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd

import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

import numpy as np

import torch
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

from sklearn.preprocessing import *

from my_dataset_1by1 import *
import os
import time
import matplotlib.pyplot as plt



FileNotFoundError: Could not find module 'C:\ProgramData\Anaconda3\Lib\site-packages\torch_sparse\_version_cpu.pyd' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
class GCN_regression_Net_single(torch.nn.Module):
    """
    feed the data one-by-one version.
    """
    def __init__(self, n_features, nhid1, nhid2, nhid3):
        super(GCN_regression_Net_single, self).__init__()
        self.conv1 = GCNConv(n_features, nhid1)
        self.conv2 = GCNConv(nhid1, nhid2)
        self.conv3 = GCNConv(nhid2, nhid3)
        #self.bn = torch.nn.BatchNorm1d(n_nodes)
        #self.dropout = dropout
        #self.linear1 = torch.nn.Linear(nhid3, 1)    #used in previous patch, reduce nhid first
        self.linear1 = torch.nn.Linear(nhid3, 1)
        self.linear2 = torch.nn.Linear(508, 1)
        #self.batch_size = batch_size
        #self.maxpool = torch.nn.MaxPool2d(4, stride = 4) # max pooling with square window of size=3, stride=2
        #no maxpooling because it will reduce the feature channel.
        
    def forward(self, X, edge_index, edge_weight):
        x, edge_index, edge_weight = X, edge_index, edge_weight #X has shape[batch_size * n_node]

        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        
        x = F.dropout(x, training=self.training)
        
        x = self.conv2(x, edge_index, edge_weight)
        x = F.relu(x)
        
        x = self.conv3(x, edge_index, edge_weight)    # X in shape: [n_nodes * batch_size, nhid3]
        x = F.relu(x)
        
        x = F.dropout(x, training=self.training) 
        
        x = self.linear1(x) #output shape in [508,1]  
        
        #print("the output layer shape is:")
        #print(x.shape)
        
        
        x = torch.squeeze(x)    #output shape in [508]
        #print(type(x))
        x = self.linear2(x) #output shape in [1]
        x = torch.squeeze(x)
        #print(x.shape)
        #x = torch.reshape(x, (self.batch_size, -1)) # output shape in [batch_size, n_node]
        return x  

In [ ]:
np.random.seed(1)
torch.random.manual_seed(1)
device = torch.device('cpu')

dataset_size = 3000
perm = torch.randperm(dataset_size).numpy()
partition = {}
partition["train"] = perm[:int(dataset_size*8/10)]
partition["validation"] = perm[int(dataset_size*8/10):int(dataset_size*9/10)]
partition["test"] = perm[int(dataset_size*9/10):]

model = GCN_regression_Net_single(n_features = 6, 
                               nhid1 = 32, 
                               nhid2 = 64, 
                               nhid3 = 128).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1E-6, weight_decay=5e-4)
loss_func = torch.nn.MSELoss()



earlystopping = True
best_epoch_num = 0
best_loss_valid = 1e10
patient = 100
prefix = "gcn_net_trained_18_single_MSELoss"
max_epochs = 500
    
train_set = torch.utils.data.Subset(My_dataset(), partition["train"])
validation_set = torch.utils.data.Subset(My_dataset(), partition["validation"])
test_set = torch.utils.data.Subset(My_dataset(), partition["test"])

In [ ]:
#model.linear2.weight

In [ ]:
#here we load the data and trained GCN model.

if __name__ == "__main__":
    
    hook_outputs = []
    total_outputs = []
    total_labels = []
    
    PATH = './models/gcn_net_trained_18_single_MSELoss.pt.final'
    model.load_state_dict(torch.load(PATH))
       
    
    #define a hook to strip embedding in each batch.
    def hook(module, input, output):
        print(module)
        hook_outputs.append(input)
        print("printing hook output")
        print(hook_outputs[0][0])
        print(hook_outputs[0][0].shape)
        #print(type(hook_outputs))
    
    #going through the selected dataset, without the gradient.(no change on model)
    h = model.linear2.register_forward_hook(hook)
    
    with torch.no_grad():
        for i, datapoint in enumerate(test_set):
            X, edge_index, edge_weight, y = datapoint.X, datapoint.edge_index, datapoint.edge_attr, datapoint.y
            
            #transfer to GPU
            #x, edge_index, edge_attr, y = Variable(torch.as_tensor(x)).to(device), Variable(torch.as_tensor(edge_index)).to(device), Variable(torch.as_tensor(edge_attr)).to(device), Variable(torch.as_tensor(y)).to(device)
            
            X, edge_index, edge_weight = Variable(torch.as_tensor(X, dtype=torch.float32)).to(device), edge_index.to(device), edge_weight.to(device)
            #y = y.to(device)
            y = Variable(torch.as_tensor(y, dtype=torch.float32)).to(device)
            
            #optimizer.zero_grad()
            
            prediction_train = model(X, edge_index, edge_weight) #forward 
            loss = loss_func(prediction_train, y)
        
            #loss.backward() # backward
            #optimizer.step() # optimizer step
            #optimizer.zero_grad()
            if i == 0:
                print("hooking datapoint no. %d" %i)
                #print(np.expand_dims(hook_outputs[0][0].detach().numpy(), axis = 1).shape)
                total_outputs = np.expand_dims(hook_outputs[0][0].detach().numpy(), axis = 1)
                total_labels.append(y.item())
                #print(total_labels.shape)
                #print(max(batch_outputs[0][0].detach().numpy()))
            else:
                print("hooking datapoint no. %d" %i)
                total_outputs = np.concatenate((total_outputs, np.expand_dims(hook_outputs[0][0].detach().numpy(), axis = 1)), axis = 1)
                print(total_outputs)
                #print(total_outputs.shape)
                total_labels.append(y.item())
                #print(total_labels.shape)
                #print(max(batch_outputs[0][0].detach().numpy()))
            #total_outputs.append(batch_outputs[0][0].detach().numpy()) #record the embedding for this batch.
            if i == 10: break
    h.remove()
    

In [ ]:
np.save("./hooked_output.npy", total_outputs.T)
np.save("./hooked_label_ref.npy", total_labels)
hooked_output = total_outputs.T

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10, svd_solver='full')
print(total_outputs)
print(len(total_labels))
pca.fit(hooked_output)
print(pca.explained_variance_ratio_)

In [ ]:
pca_result = pca.fit_transform(hooked_output)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
print(pca_result.shape)

In [ ]:
print(len(pca_result[:,0]))
print(pca_result[:,1])

In [ ]:
def lifetime_mapping(labels):
    #take in labels, numbers, output the index
    output = []
    for element in labels:
        if element < 300:
            output.append(int(0))
        elif element >=300 and element <600:
            output.append(int(1))
        elif element >=600:
            output.append(int(2))
    return output

In [ ]:
diction={'x(PC 1)': pca_result[:,0],'y(PC 2)': pca_result[:,1], 'z(PC 3)': pca_result[:,2], 'lifetime': lifetime_mapping(total_labels)}
df = pd.DataFrame(data=diction)



In [ ]:
sns.set()
sns.scatterplot(x="x(PC 1)", y="y(PC 2)", hue = 'lifetime', size = 'lifetime', data=df, alpha= 0.8)
#plt.scatter(x=pca_result[:,0], y=pca_result[:,1],alpha= 0.8)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = df['x(PC 1)']
y = df['y(PC 2)']
z = df['z(PC 3)']

ax.set_xlabel("PC 1")
ax.set_ylabel("PC 2")
ax.set_zlabel("PC 3")
from matplotlib.colors import ListedColormap
cmap = ListedColormap(sns.color_palette("husl", 256).as_hex())

ax.scatter(x, y, z, c=df['lifetime'], cmap=cmap)

plt.show()

In [ ]:
"""
for name, param in model.named_parameters():
    #if name == 'linear1.weight':
    if name == 'conv3.weight':
        #print(param.detach().numpy())
        weights = param.detach().numpy()
"""

In [ ]:
#T-SNE visualization.
from sklearn.manifold import TSNE

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)


In [ ]:
total_outputs.shape[1]

In [ ]:
tsne_results = tsne.fit_transform(total_outputs)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))


In [ ]:
diction={'x': tsne_results[:,0],'y': tsne_results[:,1], 'lifetime': lifetime_mapping(total_labels)}
df2 = pd.DataFrame(data=diction)

In [ ]:

plt.figure(figsize=(16,10))
sns.scatterplot(
    x='x', y='y',
    #hue="lifetime",
    #palette=sns.color_palette("hls", 10),
    data=df2,
    legend="full",
    alpha=0.3
)